In [2]:
import re

import polars as pl

In [8]:
cols = (
    "qname",
    "qlen",
    "qst",
    "qend",
    "ort",
    "tname",
    "tlen",
    "tst",
    "tend",
    "matches",
    "aln_blk_len",
    "q",
    "id",
    "cg",
)

In [4]:
# Get contig length.
MISASSEMBLIES = ["gap", "false-duplication", "misjoin"]
df_lens = pl.concat(
    pl.read_csv(
        f"../../results/misasim/HG00171_chr9_haplotype2-0000142_{m}_asm.fa.fai",
        separator="\t",
        has_header=False,
    )
    .rename({"column_1": "ctg", "column_2": "length"})
    .select("ctg", "length")
    .with_columns(misassembly=pl.lit(m))
    for m in MISASSEMBLIES
)
df_lens

ctg,length,misassembly
str,i64,str
"""unassigned-0000201""",64707,"""gap"""
"""unassigned-0000202""",139976,"""gap"""
"""unassigned-0000214""",42457,"""gap"""
"""unassigned-0000248""",155595,"""gap"""
"""unassigned-0000249""",168088,"""gap"""
…,…,…
"""unassigned-0003439|unassigned|…",25993,"""misjoin"""
"""unassigned-0003440|unassigned|…",166868,"""misjoin"""
"""unassigned-0003441|unassigned|…",43321,"""misjoin"""


In [11]:
# Read all paf files, rename cols, and filter only for primary alignments.
add_ctg_coord_cols = [
    "qname_ctg_start",
    "tname_ctg_start",
    "qname_ctg_end",
    "tname_ctg_end",
]
infile = "../../results/misasim/repair/HG00171_chr9_haplotype2-0000142_false-duplication_fwd_HG00171_chr9_haplotype2-0000142_gap_fwd_break.paf"
mtch = re.search(
    r"results/misasim/repair/HG00171_chr9_haplotype2-0000142_(.*?)_(.*?)_HG00171_chr9_haplotype2-0000142_(.*?)_(.*?)_break.paf",
    infile,
)
target_mtype, target_ort, query_mtype, query_ort = mtch.groups()
print(mtch.groups())

df = (
    pl.read_csv(
        infile,
        separator="\t",
        has_header=False,
    )
    .rename({f"column_{i}": c for i, c in enumerate(cols, 1)})
    .with_columns(
        target_mtype=pl.lit(target_mtype),
        query_mtype=pl.lit(query_mtype),
        new_qname=pl.col("qname").str.split_exact(":", 1),
        new_tname=pl.col("tname").str.split_exact(":", 1),
    )
    .unnest("new_qname")
    .rename({"field_0": "new_qname", "field_1": "qname_coords"})
    .unnest("new_tname")
    .rename({"field_0": "new_tname", "field_1": "tname_coords"})
    .with_columns(
        pl.col("qname_coords").str.split_exact("-", 1),
        pl.col("tname_coords").str.split_exact("-", 1),
    )
    .unnest("qname_coords")
    .rename({"field_0": "qname_ctg_start", "field_1": "qname_ctg_end"})
    .unnest("tname_coords")
    .rename({"field_0": "tname_ctg_start", "field_1": "tname_ctg_end"})
    .cast({col: pl.Int64 for col in add_ctg_coord_cols})
    .with_columns(
        pl.col("tst") + pl.col("tname_ctg_start"),
        pl.col("tend") + pl.col("tname_ctg_start"),
        pl.col("qst") + pl.col("qname_ctg_start"),
        pl.col("qend") + pl.col("qname_ctg_start"),
    )
    .join(
        df_lens.rename({"ctg": "new_qname", "misassembly": "query_mtype"}),
        on=["new_qname", "query_mtype"],
    )
    .rename({"length": "query_ctg_len"})
    .join(
        df_lens.rename({"ctg": "new_tname", "misassembly": "target_mtype"}),
        on=["new_tname", "target_mtype"],
    )
    .rename({"length": "target_ctg_len"})
    .sort(by="qname")
    .with_row_index()
)
df

('false-duplication', 'fwd', 'gap', 'fwd')


index,qname,qlen,qst,qend,ort,tname,tlen,tst,tend,matches,aln_blk_len,q,id,cg,target_mtype,query_mtype,new_qname,qname_ctg_start,qname_ctg_end,new_tname,tname_ctg_start,tname_ctg_end,query_ctg_len,target_ctg_len
u32,str,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,str,str,str,str,str,i64,i64,str,i64,i64,i64,i64
0,"""HG00171_chr9_haplotype2-000014…",2039578,88082883,88637345,"""+""","""HG00171_chr9_haplotype2-000014…",2039578,88082883,88637345,554462,554462,60,"""id:Z:""","""cg:Z:554462=""","""false-duplication""","""gap""","""HG00171_chr9_haplotype2-000014…",88082883,90122460,"""HG00171_chr9_haplotype2-000014…",88082883,90122460,93834890,93973332
1,"""HG00171_chr9_haplotype2-000014…",2039578,88637345,88711160,"""+""","""HG00171_chr9_haplotype2-000014…",2039578,88683409,88757224,73815,73815,60,"""id:Z:""","""cg:Z:73815=""","""false-duplication""","""gap""","""HG00171_chr9_haplotype2-000014…",88082883,90122460,"""HG00171_chr9_haplotype2-000014…",88082883,90122460,93834890,93973332
2,"""HG00171_chr9_haplotype2-000014…",2039578,88711160,89516527,"""+""","""HG00171_chr9_haplotype2-000014…",2039578,88849602,89654969,805367,805367,60,"""id:Z:""","""cg:Z:805367=""","""false-duplication""","""gap""","""HG00171_chr9_haplotype2-000014…",88082883,90122460,"""HG00171_chr9_haplotype2-000014…",88082883,90122460,93834890,93973332
3,"""HG00171_chr9_haplotype2-000014…",2039578,89559365,89984019,"""+""","""HG00171_chr9_haplotype2-000014…",2039578,89697807,90122461,424654,424654,60,"""id:Z:""","""cg:Z:424654=""","""false-duplication""","""gap""","""HG00171_chr9_haplotype2-000014…",88082883,90122460,"""HG00171_chr9_haplotype2-000014…",88082883,90122460,93834890,93973332
4,"""HG00171_chr9_haplotype2-000014…",2039578,90098515,90099033,"""-""","""HG00171_chr9_haplotype2-000014…",2039578,90061379,90061897,518,518,0,"""id:Z:""","""cg:Z:178=1X79=1X31=1X23=1X49=1…","""false-duplication""","""gap""","""HG00171_chr9_haplotype2-000014…",88082883,90122460,"""HG00171_chr9_haplotype2-000014…",88082883,90122460,93834890,93973332
5,"""HG00171_chr9_haplotype2-000014…",2039578,90099033,90099317,"""-""","""HG00171_chr9_haplotype2-000014…",2039578,88100506,88100790,284,284,12,"""id:Z:""","""cg:Z:75=1X35=1X32=1X96=1X42=""","""false-duplication""","""gap""","""HG00171_chr9_haplotype2-000014…",88082883,90122460,"""HG00171_chr9_haplotype2-000014…",88082883,90122460,93834890,93973332


In [ ]:
# On broken paf

# if current row stop and next row end are equivallent, means there is some indel.
# then calculate query stop and next start. this is the indel size

In [67]:
new_rows = []
for row in df.iter_rows(named=True):
    new_rows.append((row["new_qname"], "qname", row["qst"], row["qend"]))
    try:
        next_row = df.row(row["index"] + 1, named=True)
    except pl.OutOfBoundsError:
        continue
    if (
        row["qend"] == next_row["qst"]
        and row["q"] == next_row["q"]
        and row["ort"] == next_row["ort"]
    ):
        indel_size = next_row["tst"] - row["tend"]
        # Then adjust orientation.
        if row["ort"] == "+":
            new_st, new_end = row["tend"], next_row["tst"]
        else:
            new_st, new_end = (
                row["target_ctg_len"] - row["tend"],
                row["target_ctg_len"] - next_row["tst"],
            )
        new_rows.append((row["new_tname"], "tname", new_st, new_end))

target_ctg_len = df.select("target_ctg_len").row(0)[0]
query_ctg_len = df.select("query_ctg_len").row(0)[0]
df_new_rows = (
    pl.DataFrame(
        new_rows,
        schema={
            "ctg_name": pl.String,
            "name": pl.String,
            "st": pl.Int64,
            "end": pl.Int64,
        },
    )
    # Group and aggregate to min max coords
    .with_columns(row_grp_id=pl.col("name").rle_id())
    .group_by(["row_grp_id"])
    .agg(
        pl.col("ctg_name").first(),
        pl.col("name").first(),
        pl.col("st").min(),
        pl.col("end").max(),
    )
    .sort(by="row_grp_id")
    .drop("row_grp_id")
    # Then add boundary coordinates.
    .with_row_index()
    .with_columns(
        st=pl.when(pl.col("index") == pl.col("index").first())
        .then(pl.lit(0))
        .otherwise(pl.col("st")),
        end=pl.when(pl.col("index") == pl.col("index").last())
        .then(
            pl.when(pl.col("name") == "qname")
            .then(pl.lit(query_ctg_len))
            .otherwise(pl.lit(target_ctg_len))
        )
        .otherwise(pl.col("end")),
    )
    .drop("index")
)
df_new_rows

ctg_name,name,st,end
str,str,i64,i64
"""HG00171_chr9_haplotype2-000014…","""qname""",0,88637345
"""HG00171_chr9_haplotype2-000014…","""tname""",88637345,88683409
"""HG00171_chr9_haplotype2-000014…","""qname""",88637345,88711160
"""HG00171_chr9_haplotype2-000014…","""tname""",88757224,88849602
"""HG00171_chr9_haplotype2-000014…","""qname""",88711160,93834890
